In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('__getattr__', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.ptolemaic.system import System
from everest.ptolemaic.algebra import *

In [ ]:
realm = Realm(
    null=Expressor('null'),
    succ=Expressor('succ', 1),
    )

In [ ]:
realm.succ(realm.null()) in realm